## Loading the data 

In [4]:
# import the neccesary libtraries
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")



In [5]:
districts_info = pd.read_csv("D:/10 Academy/Week8/districts_info.csv")
products_info = pd.read_csv("D:/10 Academy/products_info.csv")

## District data View

In [6]:
districts_info.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
1,2685,NaN,NaN,NaN,NaN,NaN,NaN
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000["
3,3188,NaN,NaN,NaN,NaN,NaN,NaN
4,2238,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
districts_info.shape

(233, 7)

In [9]:
# a look at the percentage of the  missing data
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The district dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

percent_missing(districts_info)

The district dataset contains 27.1 % missing values.


In [11]:
# columns and the missing values
 
districts_info.isna().sum()

district_id                   0
state                        57
locale                       57
pct_black/hispanic           57
pct_free/reduced             85
county_connections_ratio     71
pp_total_raw                115
dtype: int64

From the data we can observe that there are a lot of missing values with teh highest percentage fo teh missing values from the one in the 
free/ reduced column. Therefore we are going to look at a way we can make good inference at the end with this missing values. 

dropping the row for states that are not very ompactful in the data engimeering

In [12]:
#dropping nulls states 
districts_info = districts_info[districts_info.state.notna()].reset_index(drop=True)

In [13]:
districts_info.isna().sum()

district_id                  0
state                        0
locale                       0
pct_black/hispanic           0
pct_free/reduced            28
county_connections_ratio    14
pp_total_raw                58
dtype: int64

In [14]:
# fill missing with ffill method for columns 
# Forward fill. 

def fix_missing_ffill(df, col):
    df[col] = df[col].fillna(method='ffill')
    return df[col]


def fix_missing_bfill(df, col):
    df[col] = df[col].fillna(method='bfill')
    return df[col]

districts_info['pct_free/reduced'] = fix_missing_ffill(districts_info, 'pct_free/reduced')
districts_info['county_connections_ratio'] = fix_missing_ffill(districts_info, 'county_connections_ratio')
districts_info['pp_total_raw'] = fix_missing_ffill(districts_info, 'pp_total_raw')


In [15]:
districts_info.isna().sum()


district_id                 0
state                       0
locale                      0
pct_black/hispanic          0
pct_free/reduced            0
county_connections_ratio    0
pp_total_raw                0
dtype: int64

In [17]:
for i in ['pct_black/hispanic', 'pct_free/reduced']:
    districts_info[i] = districts_info[i].apply(lambda x: float(x.split(',')[0][1:]) + 0.1)

districts_info['pp_total_raw'] = districts_info['pp_total_raw'].apply(lambda x: int(x.split(',')[0][1:]) + 1000)

districts_info.drop('county_connections_ratio', axis = 1, inplace = True)

districts_info.head(10)

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,8815,Illinois,Suburb,0.1,0.1,15000
1,4921,Utah,Suburb,0.1,0.3,7000
2,5987,Wisconsin,Suburb,0.1,0.1,11000
3,3710,Utah,Suburb,0.1,0.5,7000
4,7177,North Carolina,Suburb,0.3,0.3,9000
5,9812,Utah,Suburb,0.1,0.3,7000
6,6584,North Carolina,Rural,0.5,0.7,9000
7,1044,Missouri,Suburb,0.1,0.1,11000
8,7457,Washington,City,0.5,0.5,13000
9,1904,Connecticut,Rural,0.1,0.1,13000


In [9]:
products_info.head()

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC - Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC - Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,"LC - Sites, Resources & Reference - Games & Simulations"
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC - Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC - Digital Learning Platforms


## Engagement data 

Look at the engagement

In [4]:
engagement_dict = {
    "time":	"date in \"YYYY-MM-DD\"",
    "lp_id" :	"The unique identifier of the product",
    "pct_access" :	"Percentage of students in the district have at least one page-load event of a given product and on a given day",
    "engagement_index"	: "Total page-load events per one thousand students of a given product and on a given day"
}
pd.DataFrame.from_dict(engagement_dict, orient="index", columns=["description"])

,description
time,"date in ""YYYY-MM-DD"""
lp_id,The unique identifier of the product
pct_access,Percentage of students in the district have at...
engagement_index,Total page-load events per one thousand studen...


A look at the district Data

In [5]:
pd.set_option('display.max_colwidth', None)  
districts_info_desc = {
    "district_id"	: "The unique identifier of the school district",
    "state" :	"The state where the district resides in",
    "locale" :	"NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information.",
    "pct_black/hispanic" :	"Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data",
    "pct_free/reduced" : 	"Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data",
    "countyconnectionsratio" :	"ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.",
    "pptotalraw" :	"Per-pupil total expenditupd.set_option('display.max_rows', 500)re (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district."
}
pd.DataFrame.from_dict(districts_info_desc, orient="index", columns=["description"])

,description
district_id,The unique identifier of the school district
state,The state where the district resides in
locale,"NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information."
pct_black/hispanic,Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data
pct_free/reduced,Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data
countyconnectionsratio,ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.
pptotalraw,"Per-pupil total expenditupd.set_option('display.max_rows', 500)re (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district."


## Product Information 

A look at the product information

In [8]:
products_info_desc = {
    "LP ID"	: "The unique identifier of the product",
    "URL":	"Web Link to the specific product",
"Product Name" :	"Name of the specific product",
"Provider/Company Name" :	"Name of the product provider",
"Sector(s)"	: "Sector of education where the product is used",
"Primary Essential Function" :	"The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled"
}
pd.DataFrame.from_dict(products_info_desc, orient="index", columns=["description"])

,description
LP ID,The unique identifier of the product
URL,Web Link to the specific product
Product Name,Name of the specific product
Provider/Company Name,Name of the product provider
Sector(s),Sector of education where the product is used
Primary Essential Function,"The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled"


In [19]:
products_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   LP ID                       372 non-null    int64 
 1   URL                         372 non-null    object
 2   Product Name                372 non-null    object
 3   Provider/Company Name       371 non-null    object
 4   Sector(s)                   352 non-null    object
 5   Primary Essential Function  352 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.6+ KB


In [21]:
products_info.isna().sum()

LP ID                          0
URL                            0
Product Name                   0
Provider/Company Name          1
Sector(s)                     20
Primary Essential Function    20
dtype: int64

We seperate the primary essential for a better understatnding of the data 

In [27]:
products_info['Sector(s)'] = fix_missing_ffill(products_info, 'Sector(s)')
products_info['Primary Essential Function'] = fix_missing_ffill(products_info, 'Primary Essential Function')
# Use of the ffill
products_info['primary_function_main'] = products_info['Primary Essential Function'].apply(lambda x: x.split(' - ')[0] if x == x else x)
products_info['primary_function_sub'] = products_info['Primary Essential Function'].apply(lambda x: x.split(' - ')[1] if x == x else x)

products_info['primary_function_sub'] = products_info['primary_function_sub'].replace({'Sites, Resources & References' : 'Sites, Resources & Reference'})
products_info.drop("Primary Essential Function", axis=1, inplace=True)


products_info

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),primary_function_main,primary_function_sub
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC,Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,LC,"Sites, Resources & Reference"
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC,Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC,Digital Learning Platforms
...,...,...,...,...,...,...,...
367,88065,https://dochub.com/,DocHub,DocHub,PreK-12; Higher Ed; Corporate,SDO,Other
368,37805,http://google.com/slides/about/,Google Slides,Google LLC,PreK-12; Higher Ed; Corporate,LC,Content Creation & Curation
369,32555,http://www.innersloth.com/gameAmongUs.php,Among Us,InnerSloth,PreK-12; Higher Ed,LC,"Sites, Resources & Reference"
370,87841,http://edpuzzle.com,Edpuzzle - Free (Basic Plan),EDpuzzle Inc.,PreK-12; Higher Ed,LC,"Sites, Resources & Reference"


In [28]:
products_info.isna().sum()

LP ID                    0
URL                      0
Product Name             0
Provider/Company Name    1
Sector(s)                0
primary_function_main    0
primary_function_sub     0
dtype: int64

In [30]:
products_info = products_info.dropna()

In [31]:
products_info.isna().sum()

LP ID                    0
URL                      0
Product Name             0
Provider/Company Name    0
Sector(s)                0
primary_function_main    0
primary_function_sub     0
dtype: int64

In [34]:

merged_data = pd.merge(products_info, engagement_dict, left_on = 'LP ID', right_on = 'lp_id')
merged_data['district_id'] = merged_data['district_id'].astype('int64')
merged_data = pd.merge(merged_data, district, on = 'district_id')
merged_data.head(3)

NameError: name 'engagement_dict' is not defined